# Task Agent

Core idea of this agent is to manage tasks. Give information about existing tasks and analize tasks that already done.

In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
from typing import Dict, Any, Optional, List

In [2]:
# Load environment variables
load_dotenv() 

# Add project root to Python path and change working directory
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))
os.chdir(project_root)  # Change working directory to project root

In [3]:
from hera.task_manager import TaskManager
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

In [4]:
google_api_key = os.environ.get("GOOGLE_API_KEY", "YOUR_GOOGLE_API_KEY")
print("API Keys Set:")
print(f"Google API Key set: {'Yes' if google_api_key and google_api_key != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

API Keys Set:
Google API Key set: Yes


In [5]:
# Constants

MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

In [6]:
# Initialize Task Manager
task_manager = TaskManager()

Let's import tools from agent and tests them

In [7]:
from hera.agent import add_task, list_tasks, update_task, delete_task, list_today_tasks, list_tomorrow_tasks, list_this_week_tasks, list_next_week_tasks, list_next_month_tasks, list_tasks_for_date

def update_task_wrapper(task_id: str, title: Optional[str] = None, description: Optional[str] = None,
                timeframe: Optional[str] = None, priority: Optional[str] = None, 
                status: Optional[str] = None, due_date: Optional[str] = None,
                tags: Optional[List[str]] = None) -> Dict[str, Any]:
    return update_task(task_id, title, description, timeframe, priority, status, due_date, tags)


Example of tool usage:

In [8]:
task_result = add_task("Buy groceries", "Buy fruits, vegetables, and bread", "day", "medium", "2023-01-01", ["shopping", "groceries"])
print(f"Created a task:")
display(task_result)
print(f"Existing tasks:")
tasks_list = list_tasks(timeframe=None, status="pending", target_date=None, include_stats=False)['tasks']
display(tasks_list)

Created a task:


{'status': 'success',
 'message': 'Task added successfully',
 'task': {'id': 'b23a4a7c-bd9e-4d33-aea1-624f4be93f91',
  'title': 'Buy groceries',
  'description': 'Buy fruits, vegetables, and bread',
  'timeframe': 'day',
  'priority': 'medium',
  'status': 'pending',
  'due_date': '2023-01-01',
  'tags': ['shopping', 'groceries'],
  'created_at': '2025-09-24T10:49:26.362351',
  'updated_at': '2025-09-24T10:49:26.362357'}}

Existing tasks:


[{'id': 'f000fec0-d147-4e71-8865-5ec81b2defd3',
  'title': 'Buy groceries',
  'description': 'Buy fruits, vegetables, and bread',
  'timeframe': 'day',
  'priority': 'medium',
  'status': 'pending',
  'due_date': '2023-01-01',
  'tags': ['shopping', 'groceries'],
  'created_at': '2025-09-23T21:16:24.026989',
  'updated_at': '2025-09-23T21:16:24.026996'},
 {'id': 'b23a4a7c-bd9e-4d33-aea1-624f4be93f91',
  'title': 'Buy groceries',
  'description': 'Buy fruits, vegetables, and bread',
  'timeframe': 'day',
  'priority': 'medium',
  'status': 'pending',
  'due_date': '2023-01-01',
  'tags': ['shopping', 'groceries'],
  'created_at': '2025-09-24T10:49:26.362351',
  'updated_at': '2025-09-24T10:49:26.362357'},
 {'id': '13bb0547-b766-4e0f-bdd2-197648ec08e2',
  'title': 'Send Post',
  'description': '',
  'timeframe': 'day',
  'priority': 'high',
  'status': 'pending',
  'due_date': '2025-09-25',
  'tags': [],
  'created_at': '2025-09-24T10:24:05.138305',
  'updated_at': '2025-09-24T10:24:05.1

In [9]:
task_id = task_result['task']['id']
if task_id is not None:
    print(f"Updating task: {task_id} to completed status")
    updated_task_response = update_task_wrapper(
        task_id = task_id, 
        status = "completed", 
    )
    display(updated_task_response['task'])
    print(delete_task(task_id))
    print(f"Deleted task: {task_id}")

Updating task: b23a4a7c-bd9e-4d33-aea1-624f4be93f91 to completed status


{'id': 'b23a4a7c-bd9e-4d33-aea1-624f4be93f91',
 'title': 'Buy groceries',
 'description': 'Buy fruits, vegetables, and bread',
 'timeframe': 'day',
 'priority': 'medium',
 'status': 'completed',
 'due_date': '2023-01-01',
 'tags': ['shopping', 'groceries'],
 'created_at': '2025-09-24T10:49:26.362351',
 'updated_at': '2025-09-24T10:49:26.368545'}

{'status': 'success', 'message': 'Task deleted successfully', 'deleted_task_id': 'b23a4a7c-bd9e-4d33-aea1-624f4be93f91', 'deleted_task': {'id': 'b23a4a7c-bd9e-4d33-aea1-624f4be93f91', 'title': 'Buy groceries', 'description': 'Buy fruits, vegetables, and bread', 'timeframe': 'day', 'priority': 'medium', 'status': 'completed', 'due_date': '2023-01-01', 'tags': ['shopping', 'groceries'], 'created_at': '2025-09-24T10:49:26.362351', 'updated_at': '2025-09-24T10:49:26.368545'}}
Deleted task: b23a4a7c-bd9e-4d33-aea1-624f4be93f91


Tools seems to work fine, so let's try to integrate them into agent

## Session and Runner

- **SessionService**: Responsible for managing conversation history and state for different users and sessions. The InMemorySessionService is a simple implementation that stores everything in memory, suitable for testing and simple applications. It keeps track of the messages exchanged. We'll explore state persistence more in Step 4.

- **Runner**: The engine that orchestrates the interaction flow. It takes user input, routes it to the appropriate agent, manages calls to the LLM and tools based on the agent's logic, handles session updates via the SessionService, and yields events representing the progress of the interaction.

In [10]:
from hera.agent import root_agent

In [11]:
session_service = InMemorySessionService()

# Define constants for identifying the interaction context
APP_NAME = "task_agent_app_test"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

runner = Runner(
    agent=root_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='task_agent_app_test', User='user_1', Session='session_001'
Runner created for agent 'hera'.


Since LLM calls and tool execution can take some time, Runner operates asynchronously. Thus we need an async function to call our agent.

In [12]:
async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can comment the line below if you do not want to see *all* events during execution
      print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

In [13]:
await call_agent_async("Hi I want to create task \"Send Post\" about sending post to new customers. This is daily task that have to be done tomorrow at 8 PM. It is high priority",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)


>>> User Query: Hi I want to create task "Send Post" about sending post to new customers. This is daily task that have to be done tomorrow at 8 PM. It is high priority


  [Event] Author: hera, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'expression': 'tomorrow'
    },
    id='adk-a60a31dc-df11-406b-b979-445cf7001411',
    name='parse_date_expression'
  )
)] role='model'
  [Event] Author: hera, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='adk-a60a31dc-df11-406b-b979-445cf7001411',
    name='parse_date_expression',
    response={
      'date': '2025-09-25',
      'datetime': '2025-09-25',
      'message': 'Tomorrow: 2025-09-25',
      'status': 'success',
      'suggested_timeframe': 'day'
    }
  )
)] role='user'


  [Event] Author: hera, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'description': 'Send post to new customers',
      'due_date': '2025-09-25',
      'priority': 'high',
      'timeframe': 'day',
      'title': 'Send Post'
    },
    id='adk-f9052d04-2ad3-4432-94b4-196f89bed96b',
    name='add_task'
  )
)] role='model'
  [Event] Author: hera, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='adk-f9052d04-2ad3-4432-94b4-196f89bed96b',
    name='add_task',
    response={
      'error': """Invoking `add_task()` failed as the following mandatory input parameters are not present:
tags
You could retry calling this tool, but it is IMPORTANT for you to provide all the mandatory parameters."""
    }
  )
)] role='user'


  [Event] Author: hera, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args=<dict len=6>,
    id='adk-38e2be9c-fd8d-4dc7-a493-f18d55b0ee21',
    name='add_task'
  )
)] role='model'
  [Event] Author: hera, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='adk-38e2be9c-fd8d-4dc7-a493-f18d55b0ee21',
    name='add_task',
    response={
      'message': 'Task added successfully',
      'status': 'success',
      'task': <dict len=10>
    }
  )
)] role='user'
  [Event] Author: hera, Type: Event, Final: True, Content: parts=[Part(
  text="""OK. I have created the task "Send Post" with the description "Send post to new customers" due tomorrow, September 25, 2025. It is a high-priority daily task.
```json
{
  "status": "success",
  "message": "Task created successfully",
  "task": {
    "title": "Send Post",
    "description": "Send post to new customers",
    "due_date": "2025-09-25",
    "priority": "high",
    "

In [ ]:
list_tasks(None, None, None, False)

{'status': 'success',
 'tasks': [{'id': '839c4c78-f462-4420-9e79-5f40d6263e9f',
   'title': 'Send Post',
   'description': 'Send post to new customers',
   'timeframe': 'day',
   'priority': 'high',
   'status': 'pending',
   'due_date': '2025-09-25',
   'tags': [],
   'created_at': '2025-09-24T10:49:31.855346',
   'updated_at': '2025-09-24T10:49:31.855352'}],
 'count': 1,
 'filters': {'timeframe': None, 'status': None, 'target_date': '2025-09-24'}}

In [17]:
await call_agent_async("What tasks I have for tomorrow?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)


>>> User Query: What tasks I have for tomorrow?


  [Event] Author: hera, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={},
    id='adk-b4605f8e-89a6-44fb-924d-c9a2cfc1d158',
    name='list_tomorrow_tasks'
  )
)] role='model'
  [Event] Author: hera, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='adk-b4605f8e-89a6-44fb-924d-c9a2cfc1d158',
    name='list_tomorrow_tasks',
    response={
      'error': """Invoking `list_tomorrow_tasks()` failed as the following mandatory input parameters are not present:
status
You could retry calling this tool, but it is IMPORTANT for you to provide all the mandatory parameters."""
    }
  )
)] role='user'


  [Event] Author: hera, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'status': 'pending'
    },
    id='adk-5f08d1a6-a9a3-40fa-b911-e9025cf76390',
    name='list_tomorrow_tasks'
  )
)] role='model'
  [Event] Author: hera, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='adk-5f08d1a6-a9a3-40fa-b911-e9025cf76390',
    name='list_tomorrow_tasks',
    response={
      'count': 1,
      'filters': {
        'status': 'pending',
        'target_date': '2025-09-25',
        'timeframe': 'day'
      },
      'status': 'success',
      'tasks': [
        <dict len=10>,
      ]
    }
  )
)] role='user'
  [Event] Author: hera, Type: Event, Final: True, Content: parts=[Part(
  text="""OK. Tomorrow, September 25, 2025, you have one pending task: "Send Post" with the description "Send post to new customers". It is a high-priority daily task.
```json
{
  "status": "success",
  "count": 1,
  "tasks": [
   

<<< Agent Response: OK. Tomorrow, September 25, 2025, you have one pending task: "Send Post" with the description "Send post to new customers". It is a high-priority daily task.
```json
{
  "status": "success",
  "count": 1,
  "tasks": [
    {
      "title": "Send Post",
      "description": "Send post to new customers",
      "due_date": "2025-09-25",
      "priority": "high",
      "timeframe": "day",
      "status": "pending"
    }
  ]
}

Example of building conversation:

In [43]:
# We need an async function to await our interaction helper
async def run_conversation():
    await call_agent_async("Hi I want to create task \"Send Post\". This is daily task that have to be done tomorrow at 8 PM. It is really important",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

    await call_agent_async("What tasks I have for today?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

    await call_agent_async("What tasks I have for tomorrow?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

# Execute the conversation using await in an async context (like Colab/Jupyter)
await run_conversation()


>>> User Query: Hi I want to create task "Send Post". This is daily task that have to be done tomorrow at 8 PM. It is really important
  [Event] Author: hera, Type: Event, Final: True, Content: parts=[Part(
  text="""```json
{
  "response": {
    "status": "success",
    "message": "OK. I can create this task for you. What description and tags would you like to add?",
    "next_action": "provide_task_details"
  }
}
```"""
)] role='model'
<<< Agent Response: ```json
{
  "response": {
    "status": "success",
    "message": "OK. I can create this task for you. What description and tags would you like to add?",
    "next_action": "provide_task_details"
  }
}
```

>>> User Query: What tasks I have for today?


  [Event] Author: hera, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={},
    id='adk-7498a21b-c0e7-4b89-bbc9-794233e97b59',
    name='list_today_tasks'
  )
)] role='model'
  [Event] Author: hera, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='adk-7498a21b-c0e7-4b89-bbc9-794233e97b59',
    name='list_today_tasks',
    response={
      'error': """Invoking `list_today_tasks()` failed as the following mandatory input parameters are not present:
status
You could retry calling this tool, but it is IMPORTANT for you to provide all the mandatory parameters."""
    }
  )
)] role='user'
  [Event] Author: hera, Type: Event, Final: True, Content: parts=[Part(
  text="""```json
{
  "response": {
    "status": "error",
    "message": "To list today's tasks, I need to know what status you are looking for. Do you want tasks that are pending, in_progress, completed, or cancelled?",
    "error_code": "missing_par

  [Event] Author: hera, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={},
    id='adk-d491a03c-30f3-4a61-a050-9f3381620848',
    name='list_tomorrow_tasks'
  )
)] role='model'
  [Event] Author: hera, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='adk-d491a03c-30f3-4a61-a050-9f3381620848',
    name='list_tomorrow_tasks',
    response={
      'error': """Invoking `list_tomorrow_tasks()` failed as the following mandatory input parameters are not present:
status
You could retry calling this tool, but it is IMPORTANT for you to provide all the mandatory parameters."""
    }
  )
)] role='user'
  [Event] Author: hera, Type: Event, Final: True, Content: parts=[Part(
  text="""```json
{
  "response": {
    "status": "error",
    "message": "To list tomorrow's tasks, I need to know what status you are looking for. Do you want tasks that are pending, in_progress, completed, or cancelled?",
    "error_code": 